In [ ]:
import math
import seaborn as sns
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.metrics import accuracy_score
import random
import numpy as np
def func_en(arr=[]):
    x= (6*arr[0]+7*arr[1]+3*arr[2]+7*arr[3]+3*arr[4])*(math.sin(math.pi*arr[5]/24))
    return 1 if x>=0 else 0


In [ ]:
# 训练集
trainFile=open("c:\\ctf\\t.txt",'r').readlines()
tx=[]
ty=[]
for i in trainFile:
    try:
        s=i.strip().split('\t')
        ty.append(int(s[0]))
        tx.append(eval(s[1]))
    except:
        print("err:",i)

#训练集
t_x = tx[:39000]
t_y = ty[:39000]
t_y = np.array(t_y)
# 测试集
p_x = tx[1000:]
p_y = ty[1000:]
p_y = np.array(p_y)

#flag数据
import cv2

img=cv2.imread("c:\\ctf\\flag.png",1)
iq_y=img.flatten()[1::3]//200
q_x=[]
q_y=[]
t= [random.randint(-64,64) for i in range(6)]
import tqdm.notebook as tq

for i in tq.tqdm(iq_y):
    while func_en(t) != i:t= [random.randint(-64,64) for i in range(6)]
    q_x.append(t)
    t= [random.randint(-64,64) for i in range(6)]
q_y=np.array(iq_y)   


trainFile=open("c:/ctf/flag.txt",'r').readlines()
for i in trainFile:
    try:
        q_x.append(eval(i))
    except:
        print("err:",i)
print(q_x)



tr=''
for i in q_x: 
    tr+=str(i)
    tr+='\n'

ft=open("c:\\ctf\\flag.txt",'w')
ft.write(tr)
ft.close()  

print(len(t_y[t_y==0]),len(t_y[t_y>0]))
print(len(p_y[p_y==0]),len(p_y[p_y>0]))
print(len(q_y[q_y==0]),len(q_y[q_y==1]))
print(q_y)


In [ ]:
import matplotlib.pyplot as plt
img=q_y.reshape(79,991)
plt.figure(figsize=(20,2))
plt.imshow(img)
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn import svm
from sklearn.metrics import classification_report
# clf_rbf = svm.SVC(decision_function_shape="ovo", kernel="rbf")
# clf_rbf.fit(t_x, t_y)
# y_test_pre_rbf = clf_rbf.predict(p_x)


# print('pre_rbf')
# print(classification_report(p_y, y_test_pre_rbf))

In [ ]:
# y_test_pre_rbf = clf_rbf.predict(q_x)
# img=(y_test_pre_rbf).reshape(123,1150)#图像长宽，分解因数可得到
# plt.figure(figsize=(20,20))
# plt.imshow(img)
# plt.show()

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.metrics import accuracy_score
in_V=6
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        #输入层
        self.lin = nn.Linear(in_V, in_V*64)        
        #全连接层
        self.h0= nn.Linear(in_V*64, in_V*64)
        self.h1= nn.Linear(in_V*64, in_V*64)
        self.h2 = nn.Linear(in_V*64, 2)
        #输出层
        self.lout = nn.Softmax(dim=1)
        self.drop=nn.Dropout(p=0.2)
        
    def forward(self, x):
        x = F.sigmoid(self.lin(x))
        x=self.drop(x)
        x=F.sigmoid(self.h0(x))
        x=self.drop(x)
        x=F.sigmoid(self.h1(x))
        x=self.drop(x)
        x=F.sigmoid(self.h2(x))
        x=self.drop(x)
        return F.relu(self.lout(x))
    


net = Net()# 实例化神经网络

In [ ]:
t_x = torch.tensor(np.array(t_x[::-1]), dtype=torch.float)
t_y= torch.tensor(np.array(t_y[::-1]), dtype=torch.long)
q_x = torch.tensor(np.array(q_x), dtype=torch.float)
p_x = torch.tensor(np.array(p_x), dtype=torch.float)
q_y= torch.tensor(np.array(q_y), dtype=torch.long)
p_y= torch.tensor(np.array(p_y), dtype=torch.long)

In [ ]:
device =torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

optimizer = optim.Adam(net.parameters(), lr=0.001)  # 据说adam算法比较时髦，且好用
#optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)
criterion = nn.CrossEntropyLoss()# 交叉熵损失函数
import tqdm.notebook as td
error=[]
zql=[]
net.to(device)

net.drop=nn.Dropout(p=0.1)

for epoch in td.trange(3000):
    optimizer.zero_grad()  #梯度清零
    y_pred = net(t_x.to(device))  #反馈
    loss = criterion(y_pred, t_y.to(device)) #损失计算
    loss.backward()#
    optimizer.step()
    if epoch % 200==0:
        error.append(loss.item())
        y_pred = net(p_x.to(device))
        y_pred = torch.argmax(y_pred, dim=1)
        zql.append(float(accuracy_score(p_y, y_pred.cpu())))
#         y_pred = net(q_x.to(device))
#         y_pred = np.array(torch.argmax(y_pred.cpu(), dim=1))
#         img=y_pred.reshape(79,991)#图像长宽，分解因数可得到
#         plt.figure(figsize=(20,20))
#         plt.imshow(img)
#         plt.show()

from sklearn.metrics import classification_report
y_pred = net(p_x.to(device))
y_pred = torch.argmax(y_pred, dim=1)
print(classification_report(p_y, y_pred.cpu()))
torch.cuda.empty_cache()
import seaborn as sns
import matplotlib.pyplot as plt
sns.lineplot(data=error,color='red',label='loss')
sns.lineplot(data=zql,color='green',label='acc')
plt.show()



In [ ]:
torch.save(net.state_dict(),"c:/ctf/ctfshow_misc_chkin.ann")

In [ ]:

#net=Net()
#net.load_state_dict(torch.load("c:/ctf/ctfshow_misc_chkin.ann"))
net.to('cpu')
y_pred = net(q_x)
y_pred = np.array(torch.argmax(y_pred,dim=1))
torch.cuda.empty_cache()

img=y_pred.reshape(79,991)#图像长宽，分解因数可得到
plt.figure(figsize=(20,20))
plt.imshow(img)
plt.show()
print(classification_report(q_y, y_pred))